In [ ]:
from ExportDiscord import ExportDiscord

In [ ]:
ex_dis = ExportDiscord()

In [ ]:
ex_dis.create_raw_json_table("raw_guilds")
ex_dis.create_raw_json_table("raw_channels")
ex_dis.create_raw_json_table("raw_messages")

In [ ]:
import numpy as np
import pandas as pd
from pprint import pprint
import os
import glob
import json
import numpy

base_directory = "/home/paul/Projects/DiscordScraping/Colony"

# Recursively find all JSON files in the directory and its subdirectories
json_files = glob.glob(os.path.join(base_directory, '*.json'), recursive=True)

guilds = []
channels = []
messages = []
# Parts of a message
authors = {}
roles = []
attachments = []
embeds = []
stickers = []
reactions = []
mentions = []
for json_file_path in json_files:
    with open(json_file_path, 'r') as json_file:
        try:
            print(f"Processing {json_file}")
            data = json.load(json_file)
        except Exception as e:
            continue
    guilds.append(data["guild"])
    data["channel"]["guild_id"] = data["guild"]["id"]
    channels.append(data["channel"])
    for message in data["messages"]:
        if "roles" in message["author"].keys():
            if message["author"]["roles"] != []:
                for role in message["author"]["roles"]:
                    role["user_id"] = message["author"]
                    roles.append(role)
            del message["author"]["roles"]   
        authors[message["author"]["id"]] = message["author"]
        message["author"] = message["author"]["id"]
        if message["attachments"] != []:
            for attachment in message["attachments"]:
                attachment["message_id"] = message["id"]
                attachments.append(attachment)
            message["attachments"] = True
        else:
            message["attachments"] = False
        if message["embeds"] != []:
            for embed in message["embeds"]:
                embed["message_id"] = message["id"]
                if "image" not in embed.keys():
                    embed["image"] = ""
                if "footer" not in embed.keys():
                    embed["footer"] = ""
                embeds.append(embed)
            message["embeds"] = True
        else:
            message["embeds"] = False
        if "stickers" in message.keys():
            if message["stickers"] != []:
                for sticker in message["stickers"]:
                    sticker["message_id"] = message["id"]
                    stickers.append(sticker)
                message["stickers"] = True
            else:
                message["stickers"] = False
        if message["reactions"] != []:
            for reaction in message["reactions"]:
                reaction["message_id"] = message["id"]
                reactions.append(reaction)
            message["reactions"] = True
        else:
            message["reactions"] = False
        if message["mentions"] != []:
            for mention in message["mentions"]:
                mention["message_id"] = message["id"]
                mentions.append(mention)
            message["mentions"] = True
        else:
            message["mentions"] = False
        if "reference" in message.keys():
            message["reference"] = json.dumps(message["reference"])
        else:
            message["reference"] = ""
        messages.append(message)

        

In [ ]:
for guild in guilds:
    ex_dis.cur.execute('INSERT INTO raw_guilds_t (raw_json) VALUES (?)', (json.dumps(guild),))

In [ ]:
for message in messages:
    ex_dis.cur.execute('INSERT INTO raw_guilds_t (raw_json) VALUES (?)', (json.dumps(guild),))